In [ ]:
import pandas as pd

from tb_incubator.constants import set_project_base_path, ImplementCDR
from tb_incubator.input import load_param_info
from tb_incubator.utils import load_full_param_info
from tb_incubator.calibrate import tabulate_calib_results, plot_posterior_comparison

import arviz as az
az.style.use("arviz-doc")
import numpy as np

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]
image_path = project_paths["IMAGE_PATH"]
out_path = project_paths["OUTPUTS"]


In [ ]:
params = load_param_info()
params_full = load_full_param_info()

In [ ]:
file_suffix = "cdr_notif_25000d10000t_01nr"
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')
burnt_idata = idata.sel(draw=np.s_[2500:])

### Parameter traces

In [ ]:
#az.plot_trace(burnt_idata, figsize=(15, 22), compact=False, legend=False)

### Comparison of parameter posteriors to priors

In [ ]:
pp_comp = plot_posterior_comparison(burnt_idata, 0.995, covid_effects=True, xpert_improvement=True, apply_diagnostic_capacity=True,
                                    apply_cdr=ImplementCDR.ON_NOTIFICATION,
                                    color="cornflowerblue", width=15, height=15, ncols=3)
pp_comp.savefig(image_path / "pp_comp.png",dpi=300, format="png")
pp_comp

In [ ]:
from IPython.display import Markdown
results_table = tabulate_calib_results(burnt_idata, params_full)
results_table.to_csv(out_path / 'results_table_calib.csv', index=True)
results_table.columns = [f"**{col}**" for col in results_table.columns]
Markdown(results_table.to_markdown())